In [88]:
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime

engine = create_engine("sqlite:///../data/database.db", echo=False)


In [89]:
# Load source and target tables into DataFrames
df_source = pd.read_sql("SELECT * FROM Jumis_Account", engine)
df_target = pd.read_sql("SELECT * FROM B02_Accounts", engine)



In [90]:
df_source.head()

,AccountID,AccountCode,AccountName,CreateDate,UpdateDate,rn
0,305,11,Nemateriālie ieguldījumi,2021-05-11 12:52:25.493000,2021-05-11 12:52:25.443000,1
1,306,12,Pamatlīdzekļi,2021-05-11 12:52:25.493000,2021-05-11 12:52:25.443000,2
2,307,13,Ilgtermiņa finanšu ieguldījumi,2021-05-11 12:52:25.493000,2021-05-11 12:52:25.443000,3
3,308,21,Krājumi,2021-05-11 12:52:25.493000,2021-05-11 12:52:25.443000,4
4,309,22,Produktīvie un darba dzīvnieki,2021-05-11 12:52:25.493000,2021-05-11 12:52:25.443000,5


In [91]:
df_target.head()

,code,name,type_id


In [92]:
# Optional: Convert date strings to datetime if necessary
df_source['CreateDate'] = pd.to_datetime(df_source['CreateDate'])
df_source['UpdateDate'] = pd.to_datetime(df_source['UpdateDate'])

In [93]:
# Merge on Code to compare existing records
df_merged = df_source.merge(df_target, how='outer', left_on='AccountCode', right_on='code', indicator=True, suffixes=('_src', '_tgt'))


In [94]:
df_merged.head()

,AccountID,AccountCode,AccountName,CreateDate,UpdateDate,rn,code,name,type_id,_merge
0,305,11,Nemateriālie ieguldījumi,2021-05-11 12:52:25.493,2021-05-11 12:52:25.443,1,NaN,NaN,NaN,left_only
1,354,1110,Pētniecības un uzņēmuma attīstības izmaksas,2021-05-11 12:52:25.493,2021-05-11 12:52:25.443,50,NaN,NaN,NaN,left_only
2,355,1120,"Koncesijas, patenti, licences, datoru progr.",2021-05-11 12:52:25.493,2021-05-11 12:52:25.443,51,NaN,NaN,NaN,left_only
3,356,1130,Uzņēmuma nemateriālā vērtība,2021-05-11 12:52:25.493,2021-05-11 12:52:25.443,52,NaN,NaN,NaN,left_only
4,357,1180,Avansa maksājumi par nemateriāliem aktīviem,2021-05-11 12:52:25.493,2021-05-11 12:52:25.443,53,NaN,NaN,NaN,left_only


In [95]:
# Identify new records (present in source, not in target)
df_new = df_merged[df_merged['_merge'] == 'left_only'][['AccountCode', 'AccountName']]

In [96]:
df_new.head()

,AccountCode,AccountName
0,11,Nemateriālie ieguldījumi
1,1110,Pētniecības un uzņēmuma attīstības izmaksas
2,1120,"Koncesijas, patenti, licences, datoru progr."
3,1130,Uzņēmuma nemateriālā vērtība
4,1180,Avansa maksājumi par nemateriāliem aktīviem


In [97]:
# Identify deleted records (present in target, not in source)
df_deleted = df_merged[df_merged['_merge'] == 'right_only'][['code']]

In [98]:
df_deleted.head()

,code


In [99]:
# Identify potential updates
df_common = df_merged[df_merged['_merge'] == 'both']
df_updates = df_common[
    (df_common['AccountName'] != df_common['name'])
][['AccountCode', 'AccountName']]

In [100]:
df_updates.head()

,AccountCode,AccountName


In [101]:
with engine.begin() as conn:
    # Delete records
    for row in df_deleted.itertuples():
        conn.execute(text("DELETE FROM B02_Accounts WHERE code = :code"), {'code': row.code})
        
    # Insert new records
    for row in df_new.itertuples():
        conn.execute(text("""
            INSERT INTO B02_Accounts (code, name)
            VALUES (:code, :name)
        """), {
            'code': row.AccountCode,
            'name': row.AccountName
        })
       
    # Update changed records
    for row in df_updates.itertuples():
        conn.execute(text("""
            UPDATE B02_Accounts
            SET name = :name
            WHERE code = :code
        """), {
            'code': row.AccountCode,
            'name': row.AccountName
        })
    
    # conn.commit()